given a library of midi loops, meander through it, always playing the most similar unplayed loop


## functions


In [9]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    bar_length = 100
    # decimals = 1
    # percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    percent = f"{iteration:.2f}/{total:.2f}"
    filledLength = int(bar_length * iteration // total)
    bar = fill * filledLength + '-' * (bar_length - filledLength)
    print(f'\r{prefix} |{bar}| {percent} {suffix}', end = printEnd)

    if iteration == total: 
        print()

In [10]:
import pretty_midi

def normalize_midi(mid: pretty_midi.PrettyMIDI):
  """Normalize MIDI file so that the first note is at 0.0s."""
  for instrument in mid.instruments:
    if not instrument.notes:
      continue
    first_note_start = instrument.notes[0].start
    for note in instrument.notes:
      note.start -= first_note_start
      note.end -= first_note_start
  return mid

In [11]:
from scipy.spatial.distance import cosine

def find_most_similar_vector(target_vector, vector_array):
  most_similar_vector = None
  highest_similarity = -1  # since cosine similarity ranges from -1 to 1

  for vector_data in vector_array:
    name, vector = vector_data.values()
    similarity = 1 - cosine(target_vector, vector)  # type: ignore
    if similarity > highest_similarity:
      highest_similarity = similarity
      most_similar_vector = name

  return most_similar_vector

In [12]:
import mido

def play_midi_file(midi_data: mido.MidiFile, midi_port):
  t = 0
  for msg in midi_data.play():
    printProgressBar(t, midi_data.length, suffix='s')
    t += msg.time # type: ignore
    midi_port.send(msg)
  printProgressBar(midi_data.length, midi_data.length, suffix='s')

## calculate metrics


In [13]:
import os
import midi_utils as mu

midi_metrics = {}
src_file = "MIDI1-TRIMMED"
folder_path = os.path.join("data", "outputs", "time", src_file)
config = {"w1": 0.5, "w2": 0.5, "bin_length": 1.0}

for file in os.listdir(folder_path):
  if file.endswith('.mid') or file.endswith('.midi'):
    file_path = os.path.join(folder_path, file)
    midi = pretty_midi.PrettyMIDI(file_path)

    normalized_midi = normalize_midi(midi)
    metrics = mu.all_metrics(midi, config)

    midi_metrics[file] = {
      # "notes": normalized_midi,
      "metrics": metrics,
      "played": 0,
    }

In [14]:
import json

dict_file = os.path.join("data", "outputs", f"{src_file}_metrics.json")

with open(dict_file, 'w') as f:
  json.dump(midi_metrics, f)

## play


In [15]:
import time
import numpy as np
from IPython import display

# midi settings
midi_device_name = "Disklavier"
# midi_device_name = "to Max 1"
port = mido.open_output(midi_device_name) # type: ignore

# loop settings
loop_chance = 0.0 # 0-1
play_time = 6000 # seconds
time_elapsed = 0 # seconds
loop_time = 8 # seconds

# file system settings
seed = np.random.randint(0, len(midi_metrics.items()) - 1)
current_file_path = list(midi_metrics.items())[seed][0] # choose random initial file
next_file_path = None
playback_record_path = os.path.join("data", "outputs", "logs", f"pb_{src_file}_{seed}_{round(time.time(), 0)}.txt")

while (time_elapsed < play_time):
  # mark progress
  midi_metrics[current_file_path]['played'] = True
  with open(playback_record_path, 'a') as f:
    f.write(current_file_path) # type: ignore
    f.write('\n')

  # find next file
  if (np.random.rand() > loop_chance):
    cph = list(midi_metrics.items())[seed][1]['metrics']['pitch_histogram']
    all_vecs = [
      {'name': filename, 'metric': details['metrics']['pitch_histogram']}
      for filename, details in midi_metrics.items()
      if not details['played']
    ]
    next_file_path = find_most_similar_vector(cph, all_vecs)
  else:
    next_file_path = current_file_path
  print(f"[{time_elapsed:.2f}/{play_time:.2f}]\tPlaying {current_file_path} \t(next up is {next_file_path})")

  # play file
  midi = mido.MidiFile(os.path.join(folder_path, current_file_path)) # type: ignore
  time_elapsed += midi.length
  # display.display(mu.draw_midi(os.path.join(folder_path, current_file_path))) # type: ignore
  play_midi_file(midi, port)
  # display.clear_output(wait=True)

  # move to next file
  current_file_path = next_file_path


[0.00/6000.00]	Playing MIDI1-TRIMMED_152-160.mid 	(next up is MIDI1-TRIMMED_160-168.mid)
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 7.93/7.93 s
[7.93/6000.00]	Playing MIDI1-TRIMMED_160-168.mid 	(next up is MIDI1-TRIMMED_160-168.mid)
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 8.00/8.00 s
[15.93/6000.00]	Playing MIDI1-TRIMMED_160-168.mid 	(next up is MIDI1-TRIMMED_160-168.mid)
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 8.00/8.00 s
[23.93/6000.00]	Playing MIDI1-TRIMMED_160-168.mid 	(next up is MIDI1-TRIMMED_168-176.mid)
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 8.00/8.00 s
[31.94/6000.00]	Playing MIDI1-TRIMMED_168-176.mid 	(next up is MIDI1-TRIMMED_168-176.mid)
 |██████████████████████████████████████████████████████████████████████████████████████

KeyboardInterrupt: 